In [32]:
import os

In [33]:
%pwd

'd:\\Machine Learning\\Wine-Classification'

In [34]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainingConfig:
      root_dir: Path
      train_data: Path
      test_data: Path
      model: str
      n_estimators: int
      min_samples_split: int
      min_samples_leaf: int
      max_features: str
      target_column: str

In [35]:
from src.wineModel.utils import *
from src.wineModel.constants import *


class ConfigurationManager:
      def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
      ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directory([self.config.artifacts_root])

      
      def get_model_training_config(self) -> ModelTrainingConfig:
            config = self.config.model_trainer
            params = self.params.RandomForest
            schema = self.schema.TARGET_COLUMN

            create_directory([config.root_dir])

            model_train_config = ModelTrainingConfig(
                  root_dir = config.root_dir,
                  train_data = config.train_data,
                  test_data = config.test_data,
                  model = config.model,
                  n_estimators = params.n_estimators,
                  min_samples_split = params.min_samples_split,
                  min_samples_leaf = params.min_samples_leaf,
                  max_features = params.max_features,
                  target_column = schema.name
            )
            return model_train_config

In [36]:
import pandas as pd
import os
from src.wineModel import logger
from sklearn.ensemble import RandomForestRegressor
import joblib


class ModelTrainig:
      def __init__(self, config: ModelTrainingConfig):
            self.config = config
      
      def model_train(self):
            train_data = pd.read_csv(self.config.train_data)
            test_data = pd.read_csv(self.config.test_data)

            x_train = train_data.drop(columns='quality')
            x_val = test_data.drop(columns='quality')
            y_train = train_data[['quality']]
            y_val = test_data[['quality']]

            model = RandomForestRegressor(
                  n_estimators = self.config.n_estimators,
                  min_samples_split = self.config.min_samples_split,
                  min_samples_leaf = self.config.min_samples_leaf,
                  max_features = self.config.max_features
            )

            model.fit(x_train, y_train)

            joblib.dump(model, os.path.join(self.config.root_dir, self.config.model))
            logger.info("Model saved successfully")

In [37]:
try:
      config = ConfigurationManager()
      model_training_config = config.get_model_training_config()
      model_train = ModelTrainig(config=model_training_config)
      model_train.model_train()
except Exception as e:
      raise e

[2026-02-13 11:25:29,906: INFO: __init__: yaml file: config\config.yaml is loaded successfully]
[2026-02-13 11:25:29,914: INFO: __init__: yaml file: params.yaml is loaded successfully]
[2026-02-13 11:25:29,919: INFO: __init__: yaml file: schema.yaml is loaded successfully]
[2026-02-13 11:25:29,921: INFO: __init__: Directory is created at: artifacts]
[2026-02-13 11:25:29,922: INFO: __init__: Directory is created at: artifacts/model_trainer]


d:\Machine Learning\Wine-Classification\venv\Lib\site-packages\sklearn\base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[2026-02-13 11:25:30,833: INFO: 3560250092: Model saved successfully]
